<a href="https://colab.research.google.com/github/jals-code-dump/Coursera_Capstone/blob/master/Final_Assignment2_0%20-%20both%20types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

get all towns data

In [1]:
%cd C:\Users\alexc\OneDrive\Documents\GitHub\Coursera_Capstone

C:\Users\alexc\OneDrive\Documents\GitHub\Coursera_Capstone


In [ ]:
import pandas as pd
import numpy as np

towns_df = pd.read_csv('population by town-city.csv')
towns_df['Size'] = "Other"
 
for index, row in towns_df.iterrows():
  towns_df.iloc[index, 0] = towns_df.iloc[index, 0].replace("buadisp2011:", "")
  towns_df.iloc[index, 0] = towns_df.iloc[index, 0].replace(" BUASD", "")
  towns_df.iloc[index, 0] = towns_df.iloc[index, 0].replace(" BUA", "")
  string = towns_df.iloc[index, 0]
  i = 0
  i = string.find(" - ")
  if i != -1:
    towns_df.iloc[index, 0] = string[i+3:]
 
for index, row in towns_df.iterrows():
  if row['2019'] >= 7500 and row['2019'] <= 24999:
    towns_df.iloc[index, 11] = "Small Town"
  elif row['2019'] >= 25000 and row['2019'] <= 59999:
    towns_df.iloc[index, 11] = "Medium Town"


towns_df.to_csv('towns0.csv')

get long lat of all towns

In [ ]:
import pandas as pd
!pip install geocoder
import geocoder
import time

towns_df = pd.read_csv('towns0.csv', index_col=0)

towns_df['Latitude'] = ""
towns_df['Longitude'] = ""
 
 
for index, row in towns_df.iterrows():
  start_time = time.time()
  latlong = None
  if time.time() <= (start_time + 10.0):
    while latlong == None:
      g = geocoder.arcgis('{}, United Kingdom'.format(row['Area']))
      latlong = g.latlng
    lat = str(latlong[0])
    long = str(latlong[1])
    towns_df.iloc[index, 13] = long
    towns_df.iloc[index, 12] = lat

#drop empty values or re-run??

towns_df.to_csv('towns1.csv')
towns_df.head()

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


,Area,mnemonic,2011,2012,2013,2014,2015,2016,2017,2018,2019,Size,Latitude,Longitude
0,Kexby,E34000001,342,342,353,360,361,361,364,352,337,Other,53.36187000000007,-0.6923699999999258
1,Bradfield Heath,E34000002,724,715,677,702,694,757,821,838,831,Other,51.925320000000056,1.104780000000062
2,Huntley,E34000003,1108,1121,1095,1113,1119,1118,1127,1108,1131,Other,51.87339000000003,-2.409249999999929
3,Swinefleet,E34000004,787,758,789,799,819,811,787,794,770,Other,53.69073000000003,-0.8340599999999654
4,Foulsham,E34000005,1021,995,999,960,996,1022,1017,987,965,Other,52.78051000000005,1.010020000000054


search 5k within small and medium towns and save to new dataframe. changed to 20k as results were suspicious

In [ ]:
import pandas as pd
import requests
import time

towns_df = pd.read_csv('towns1.csv', index_col=0)

CLIENT_ID = '31FBQ4JXECZV0OKLQFEDFTWIKMA0CCKDTNJ3JFCMTQLNQHT3' # your Foursquare ID
CLIENT_SECRET = 'UHA3UUT4XUGCX5ABTQSH41WXLGNQUISCE35RFSFCG1HHP5LG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 20000
categoryId= '52f2ab2ebcbc57f1066b8b46'
 
# create URL
def create_url(lat, long):
  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{},&radius={}&limit={}&categoryId={}'.format(
      CLIENT_ID, 
      CLIENT_SECRET, 
      VERSION, 
      lat,
      long,
      radius, 
      LIMIT,
      categoryId)
  return url
 
 
res_df = pd.DataFrame(columns=['Area', 'Venue Category', 'Venue Name', 'Longitude', 'Latitude'])
temp_df = res_df
temp_df.at[1, :] = ""
mask_df = towns_df[towns_df['Size'] != 'Other']
for index, row in mask_df.iterrows():
  temp = temp_df
  lat = row['Latitude']
  long = row['Longitude']
  url = create_url(lat, long)
  try:
    results = requests.get(url).json()["response"]['groups'][0]['items']
    time.sleep(2)
    for res in results:
      temp['Area'] = row['Area']
      temp['Venue Name'] = res['venue']['name']
      temp['Venue Category'] = res['venue']['categories'][0]['name']
      temp['Longitude'] = res['venue']['location']['labeledLatLngs'][0]['lng']
      temp['Latitude'] = res['venue']['location']['labeledLatLngs'][0]['lat']
      res_df = res_df.append(temp, ignore_index=True)
  except:
    print("Skipping {}".format(index))

res_df.to_csv('results.csv')

verify stores are located within towns

In [ ]:
import pandas as pd
towns_df = pd.read_csv('towns1.csv', index_col=0)
res_df = pd.read_csv('results.csv', index_col=0)

from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))

LocationList = []
for index, row in towns_df.iterrows():
  LocationList.append({'Area': row['Area'], 'lat' : row['Latitude'], 'lon': row['Longitude']})
 #               {'lat': 39.7622292, 'lon': -86.1578917}]

for index, row in res_df.iterrows():
  v = {'lat': row['Latitude'], 'lon': row['Longitude']}
  area = closest(LocationList, v)
  res_df.iloc[index, 0] = area['Area']

res_df.to_csv('results1.csv')

verify store size by checking it's name against long lat result

In [ ]:
!pip install geopy
from geopy.geocoders import Nominatim
import pandas as pd
import requests
import time

%cd C:\Users\alexc\OneDrive\Documents\GitHub\Coursera_Capstone
towns_df = pd.read_csv('towns1.csv', index_col=0)
res_df = pd.read_csv('results1.csv', index_col=0)

res_df = res_df[res_df['Venue Category'] == 'Supermarket']
res_df.drop_duplicates(inplace=True)
res_df.reset_index(inplace=True, drop=True)

# create URL
def create_url(lat, long):
  url = 'https://nominatim.openstreetmap.org/reverse?&format=json&lat={}&lon={}&zoom=18&address_details=1'.format( 
      lat, 
      long)
  return url

for index, row in res_df.iterrows():
  try:
    lat = row['Latitude']
    lon = row['Longitude']
    results = requests.get(create_url(lat, lon)).json()['address']['shop']
    time.sleep(1)
    res_df.iloc[index, 3] = results
    print("Original: {}, New: {}.".format(row['Venue Name'], results))
  except:
    print("Skipping {}.".format(index))
    res_df.iloc[index, 3] = "Drop Me"

res_df = res_df[res_df['Venue Name'] != 'Drop Me']
res_df.to_csv('results2.csv')

C:\Users\alexc\OneDrive\Documents\GitHub\Coursera_Capstone
Original: Waitrose & Partners, New: Waitrose.
Original: Waitrose & Partners, New: Little Waitrose.
Skipping 2.
Original: Lidl, New: Lidl.
Original: Sainsbury's, New: Sainsbury's.
Original: Tesco, New: Tesco.
Original: Tesco, New: Tesco.
Original: Aldi, New: Aldi.
Original: Aldi, New: ALDI.
Skipping 9.
Original: Morrisons, New: Morrisons.
Original: Asda, New: Asda.
Original: Waitrose & Partners, New: Waitrose.
Original: Sainsbury's, New: Sainsbury's.
Original: Waitrose & Partners, New: Waitrose.
Original: Tesco, New: Tesco.
Original: Tesco, New: Tesco Extra.
Original: The Co-operative Food - Littleport, New: The Co-operative Food.
Skipping 18.
Original: Sainsbury's, New: Sainsbury's.
Original: Waitrose & Partners, New: Little Waitrose.
Skipping 21.
Original: Tesco, New: Tesco.
Original: Tesco, New: Tesco.
Original: Sainsbury's, New: Sainsbury's.
Original: Tesco Extra, New: Tesco Extra.
Original: Tesco Extra, New: Tesco Extra.
Or

remove duplicates and catergorise as supermarket/small supermarket

In [6]:
import pandas as pd
res_df = pd.read_csv('results2.csv', index_col=0)
res_df.drop(columns='Venue Category', inplace=True)
res_df.reset_index(inplace=True, drop=True)

#remove items that aren't supermarkets
drop_list = ['Kwik Fit','Savers', 'Room 44', "Jo's Cakes & More", 'Poundstretcher', 'Westquay North Shopping Centre',
             'Mind', 'Armada Shopping Centre', 'WHSmiths', 'Jackson Square', 'Ken Warne', 'Okehampton Pet Supermarket', 
             'Shell Select', 'ATS Euromaster', 'Arrow Auto Services', 'QD Stores', 'Gillingham Suite & Bed Centre',
             'Serene Beauty', 'Serpentine Green', 'Roys', 'WHSmith', 'Wilko', 'Cheadle Hulme Shopping Centre', 'B&M Bargains', 
             'Bathstore', 'Wickes', 'Bargain Buys', 'The Spires Shopping Centre', 'Pulse Car Audio', 'Lee Fletcher Funeral Services', 
             'Carpetright', 'B&M Stores', 'BT Blinds', 'Shell SELECT', 'ThornAlley Funeral Services', 'Barnardos', 'Oliver Bonas', 
             'Riverside Shopping Centre', 'Dunelm', 'Grains', 'Batley Shopping Centre', 'Dreams', 'Paul George', 'The Village Square', 
             'Princes Mead', 'Aujla Superstore', 'Sundon Superstore', 'Bluewater', 'Hair Design', 'Cycleworks', 'The Swan Centre', 
             'The Local Launderette', 'Ellesmere Centre', 'Crystal Peaks Shopping Centre', 'intu Merry Hill', 'Vape Store', 'Richer Sounds',
             'Marks and Spencer', 'The Range', 'Standish Pharmacy', 'The Heart Car Park', 'S I Hair', 'Argos', 'Smart Bargains', 'Esso',
             'Pets at Home', 'Debenhams', 'Halfords', 'Shell', 'Homebase', 'The Broadwalk Shopping Centre', 'Oak Furnitureland', 
             'Kingsland Shopping Centre', 'Maplin', 'The Co-operative Funeralcare', 'Shampoodles', 'Pavilions Shopping Centre', 
             'Bletchley FastFit Station', 'Family Bargains', 'Crossleys Accident Repair Centre', 'Arndale Centre', 'Grove Newsagents', 
             'Dulux Decorator Centre', 'St Nicholas Arcades', 'Wickes Extra', 'Margarets Flowers', 'The Hanger', 'Aldi (closed)', 
             'intu Uxbridge (The Chimes)', 'Brookford Launderettes', 'Sports Direct', 'Birtley News', 'John Lewis at Home', 
             'BetFred', 'Bargain Booze', 'The Salad Bowl', 'TK Maxx', 'Poundstrecher', 'bensons for beds', 'Trago', 
             'Old Parish Bakery', 'Mountain Warehouse', 'Argos Extra', 'John Lewis','Bottisham Village St','Wing Yip Centre',
             'oseyo eats Waterloo', 'Seoul Plaza', 'Vape Store Reading', 'Poundland', 'John Lewis Food Hall from Waitrose', 'Melo’s', 'Jempsons Local', 
             'Waitrose & Partners JL Foodhall', 'Ichiba', 'Loon Fung Colindale','Linda Shop', 'The Co-operative Food - Narborough Road, Leicester', 'Eat17',
              'Super Sam Derby Ltd', 'UK CBD OILS - Full Spectrum CBD', 'Shopforyu', 'Company Shop', 'Wah Yan Hong', 'W H Lung Cash & Carry',
             'Middlewood Locks Co-op', 'Dpps Trading Ltd', 'Tai Sun Oriental', 'Lemongrass Thai Supermarket in sheffield', 'Brambles', 
              'Red Orange Ltd' , 'Al Halal, Birmingham','Ken Warne Ltd','Sunrise Eggs','Jempsons Peasmarsh', 'Bexley Road Stores',
              'Korzinka Taste Of Europe', 'Haji & Sons', "Sweeney's Budgens", 'Stafford', 'Patidar Ltd', 'Derbyshire Lane West',
               'Sutton Coldfield', 'Roys Of Dereham','B&M Store', 'Longstock Farm Shop & Cafe', 'Red Chilli', 'Roys Of Beccles', 'Roys Thetford', 
               'Kashmir Superstore',  'Vape Store Redhill','Leftleys Of Wells',  'Aguona LTD','M&S Foodhall', 'Parkwood Essentials','Habers world', 'Green Valley',
             'Al-Dimashqi', 'Oseyo', 'Super Lincs', 'Waitrose Cafe', 'Trago Merthyr Tydfil', 'Magnum Stores','Megasave', 'Masala Bazaar', 'Centra Quickstop']


small_list = ['Waitrose', 'Little Waitrose', "Sainsbury's Local", 'The Co-operative Food', 'M&S Food', 'Tesco Express', 'Waitrose & Partners', 
              'Nisa Local', 'Co-op Food', 'Tesco Metro', 'Co-Op', 'Lithuanian Grocery', 'Ken Warne Licenced Grocers', 'Nisa', 'Heron Foods', 
              'Spar', 'Nisa Today', 'John Lewis', 'Waitrose Edgware Road', 'Co-op', 'M&S Foodhall', "Sainsbury's Local Chapel House", 'Tesco Esso Express',
              'Korea Foods', 'H Mart', 'The Co-operative Food - West Bromwich','TFX International Food Centre', 'Nisa Local Haldens W.G.C.', 'Infinity Foods','Waitrose & Partners at Birchanger Green  Services',
              'M&S Food To Go','Waitrose & Partners at Hartshead Services','M and S SIMPLY FOOD','Maxum Foods', 'Trident Stores Ltd', 'The Co-operative Food - Kings Lynn',
              'Co-operative Food Store', 'The Co-operative Food - Wells-next-the-Sea', 'SPAR', 'Waitrose & Partners Evesham', 
              'The Co-operative Food - Hawthorne Road, Kingstanding', 'The Co-operative Food -  Erdington', 'The Co-operative Food -  Walsall Road, Great Barr'
              'Waitrose & Partners at Michaelwood South Services','The Co-operative Food - Lowestoft''The Co-Operative Food',  'Oakham Co-op', 'The Co-operative',
              'Waitrose & Partners Swaffham','The Co-operative Food - Long Street, Atherstone', 'Waitrose & Partners at South Mimms Services South Mims', 
              'The Co-operative Food - Mulbarton', 'The Co-operative Food - Cromer','The Co-operative Food - Openwoodgate','The Co-operative Food - Castle Donington',
              'The Co-operative Food - Thurnby Lodge', 'The Co-operative Food - Carlton Colville','The Co-operative Food - Dodworth','The Co-operative Food - Birmingham Road',
              'The Co-operative Food - Bottesford', 'Coop Bobblestock','ParcMarket', 'ParcMarket Express', 'The Co-operative Food - Marston Moretaine', 'The Co-operative Food - Etching Hill',
              'The Co-operative Food - Rising Brook', 'The Co-operative Food - Sawtry','East of England Co-operative Foodstore', 'East of England Co-operative Supermarket',
              'Waitrose & Partners at Warwick North Services Warwick North','Waitrose & Partners at Membury West Services', 'Michaels Food Market''Waitrose & Partners at Warwick North Services',
              'co-op','The Co-operative Food - Dig Street, Ashbourne', 'The Co-operative Food - Belper','Waitrose & Partners Winchmore Hill', 'The Co-operative Food - Main Street, Stretton', 
              'The Co-operative Food - Stapenhill','Waitrose & Partners at Leicester Forest East - North Services','The Co-operative Food - Fleckney', 'Marks Mini Market']

clean_df = res_df
clean_df = clean_df.iloc[:, :-2] #area name and type
drop_index = []

#remove unwanted data
print("Removing unwanted Data")
for index, row in clean_df.iterrows():
  if row['Venue Name'] in drop_list:
    drop_index.append(index)
  #elif row['Venue Name'] in small_list:
    #clean_df.iloc[index, 1] = "Convenience Store" #venue name
 # else:
   # clean_df.iloc[index, 1] = "Supermarket"

#clear unwanted data from 05 system type            
clean_df.drop(index = drop_index, axis =1, inplace=True)
clean_df.reset_index(drop=True, inplace=True)
#clean_df.to_csv('results3.csv')


print(clean_df['Venue Name'].unique().tolist())
clean_df.head()

Removing unwanted Data
['Waitrose & Partners', 'Lidl', "Sainsbury's", 'Tesco', 'Aldi', 'Morrisons', 'Asda', 'The Co-operative Food - Littleport', 'Tesco Extra', 'Booths', 'Morrisons Bolsover', 'The Co-operative Food - Eckington', 'Co-op Food', 'Job Lot', 'Nisa Local', 'Tesco Esso Express', 'The Co-operative Food - Chesterfield', "Sainsbury's Local", 'Nisa Extra', 'Iceland', 'Co-op', 'The Co-operative Food - Raunds', 'The Co-operative Food - Rothwell', 'The Co-operative Food - Desborough', 'Morrisons Bedlington', 'The Food Warehouse by Iceland', 'Korea Foods', 'H Mart', 'TFX International Food Centre', 'Waitrose', 'Little Waitrose', 'Little Waitrose & Partners', 'Infinity Foods', 'Trident Stores Ltd', 'Tesco Metro', 'Waitrose & Partners at Birchanger Green  Services', 'Tesco Express', 'Morrisons Birtley', 'M&S Food To Go', 'Tesco Superstore', 'Marks Mini Market', 'Sainsbury', 'Maxum Foods', 'Co-operative Food Store', 'Loon Fung', 'Nabi Supermarket', 'Supermarket', 'The Co-operative Food

,Area,Venue Name
0,Dorchester (West Dorset),Waitrose & Partners
1,Middleyard,Waitrose & Partners
2,Dorchester (West Dorset),Lidl
3,Gamble's Green,Lidl
4,Gamble's Green,Sainsbury's


associate each supermarket to it's closest town

In [7]:
import pandas as pd
clean_df = pd.read_csv('results3.csv', index_col=0)

towns_df = pd.read_csv('towns1.csv', index_col=0)
towns_df = towns_df[towns_df['Size'] != 'Other']
towns_df.reset_index(drop=True, inplace=True)

towns_df['Convenience Store'] = 0
towns_df['Supermarket'] = 0

drop_list = []

for index, row in towns_df.iterrows():
  try:
    mask_df = clean_df[clean_df['Area'] == row['Area']]
    for i, r in mask_df.iterrows():
      if r['Venue Name'] == 'Supermarket':
        towns_df.iloc[index, -1] = towns_df.iloc[index, -1] + 1
      else:
        towns_df.iloc[index, -2] = towns_df.iloc[index, -2] + 1
  except:
    print('Error')
  if towns_df.iloc[index, -1] + towns_df.iloc[index, -2] == 0:
    drop_list.append(index)

towns_df.drop(index = drop_list, axis =1, inplace=True)
towns_df.reset_index(drop=True, inplace=True)
towns_df.to_csv('towns2.csv')

show map with markers based on relative shops per pop

In [55]:
towns_df = pd.read_csv('towns2.csv', index_col=0)
towns_df['Convenience Stores Requirement'] = 0
towns_df['Supermarkets Requirement'] = 0
sizes = ['Small Town', 'Medium Town']
ratios = []
for i, size in enumerate(sizes):
  mask_df = towns_df[towns_df['Size'] == size]
  pop = mask_df['2019'].sum()
  conv = mask_df['Convenience Store'].sum()
  sup = mask_df['Supermarket'].sum()
  print("Total of {} people, with {} convenience stores and {} supermarkets for {}s.".format(pop, conv, sup, size))
  ratios.append(conv/pop)
  ratios.append(sup/pop)
  conv2 = (conv/pop)
  sup2 = (sup/pop)
  people = 1000
  print("Per {} people a mean of {} convenience stores and {} supermarkets for {}s.".format(people, conv2*people, sup2*people, size))
  print("Ratio of {} to {} convenience stores to {} supermarkets.".format(10000, (conv/pop)*10000, (sup/pop)*10000))

#ratio of convenience stores to supermarkets
for i, size in enumerate(sizes):
  print(i)
  mask_df = towns_df[towns_df['Size'] == size]
  for index, row in mask_df.iterrows():
    towns_df.iloc[index, -2] = round(ratios[i] * row['2019'], 0) - row['Convenience Store']
    towns_df.iloc[index, -1] =  round(ratios[i+1] * row['2019'], 0) - row['Supermarket']
towns_df[towns_df['Size'] == 'Small Town']
towns_df.to_csv('towns3.csv')

Total of 6639496 people, with 143 convenience stores and 939 supermarkets for Small Towns.
Per 1000 people a mean of 0.021537779373615104 convenience stores and 0.14142639742534674 supermarkets for Small Towns.
Ratio of 10000 to 0.21537779373615104 convenience stores to 1.4142639742534675 supermarkets.
Total of 8644155 people, with 96 convenience stores and 742 supermarkets for Medium Towns.
Per 1000 people a mean of 0.011105770315317112 convenience stores and 0.08583834972880519 supermarkets for Medium Towns.
Ratio of 10000 to 0.11105770315317112 convenience stores to 0.8583834972880519 supermarkets.
0
1


visualise both types

In [59]:
!pip install folium
towns_df = pd.read_csv('towns3.csv', index_col=0)
import folium

conv_map = folium.Map(location=[55.3781, -3.4360], zoom_start=7)
for index, row in towns_df.iterrows():
  lat = row['Latitude']
  long = row['Longitude']
  tooltip = row['Area'] + ", requirement:  " + str(int(row['Convenience Stores Requirement'] + row['Supermarkets Requirement']))
  popup = "Population: " + str(row['2019']) + ", Current Stores: " + str(row['Convenience Store'] + row['Supermarket'])
  color = 'blue'
  if (row['Convenience Stores Requirement'] + row['Supermarkets Requirement']) > 0:
    color = 'green'
  elif (row['Convenience Stores Requirement'] + row['Supermarkets Requirement']) < 0:
    color = 'red'
  icon = folium.Icon(color=color)
  folium.Marker([lat, long], tooltip=tooltip, popup=popup, icon=icon).add_to(conv_map)


conv_map

population predict and estimate requirement for new stores